In [58]:
import string
import pandas as pd 
import numpy as np
import pymorphy2
import tokenize_uk as tk

from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.utils import shuffle, resample
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report

from stop_words import get_stop_words

pd.set_option('display.max_colwidth', -1)
import warnings
warnings.filterwarnings('ignore')

In [4]:
data = pd.read_parquet('../data/classification_data.parquet')

In [7]:
shuffle(data).head(10)

,subjectivity,sentence_uk,y
index,,,
18780,0.875,"Істотно, майже в два рази, з початку року по жовтень зросла вартість комунальних послуг (квартплата, електроенергія, вода, газ і т.д.).",1
10163,0.000,"""Закон Магнітського"" передбачає візові заборони і заморожування майна чиновників РФ, причетних до смерті юриста Сергія Магнітського, який помер у російській в'язниці в 2009 році.",0
139434,0.000,"У недобудованих секціях продані всі квартири, але чи завершиться будівництво - ніхто не знає.",0
35111,0.000,"Постанова Печерського районного суду м. Києва від 15.02.2017 року про закриття провадження у справі набрала законної сили"", - написав тоді його адвокат.",0
61850,0.000,Про це в ефірі ObozTV заявив народний депутат IV-VI скликання Василь Горбаль.,0
146939,0.000,"Мосійчук заявляє, що дане відео є наслідком ""жорстоких катувань"" .",0
112130,0.000,"Підтягуйтеся хто може, так починався майдан.",0
138355,0.000,"Як повідомляв ""Обозреватель"", восени 2016 року українським вчителям планують знову підвищити зарплати.",0
32640,0.950,"""Я, чесно кажучи, не знаю, що трапилося з Шевченком, можу лише сказати, щоб преса і люди дивилися на наші оголошення.",1


In [8]:
data.groupby(['y'])['sentence_uk'].count()

y
0    90803
1    16703
Name: sentence_uk, dtype: int64

In [30]:
subj = data[data['y'] == 1]
obj = data[data['y'] == 0]

obj_d = resample(obj, replace=False, n_samples=len(subj), random_state=1234)

df = shuffle(pd.concat([obj_d, subj]))

In [31]:
stop_words = get_stop_words('uk')
morph = pymorphy2.MorphAnalyzer(lang='uk')


def analyze(words): 
    result_words = []
    for w in words: 
        m = morph.parse(w)[0]
        lemma = m.normal_form
        if lemma not in stop_words and lemma not in string.punctuation: 
            result_words.append(lemma)
    return result_words


text = df['sentence_uk'].apply(tk.tokenize_words)
text = text.apply(analyze)

In [56]:
y = df['y'].values
X_train, X_test, y_train, y_test = train_test_split(text, y, test_size=0.2, random_state=1234, stratify=y)

vectorizer = CountVectorizer(analyzer='word', lowercase=False, tokenizer=lambda x: x)
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [57]:
clf = SGDClassifier(loss='log', penalty='elasticnet', n_jobs=-1, learning_rate='optimal')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
f1_score(y_test, y_pred)

0.8242791142265414

In [60]:
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.80      0.90      0.84      3341
          1       0.88      0.77      0.82      3341

avg / total       0.84      0.83      0.83      6682

